In [60]:
from dotenv import load_dotenv
import os
# from langchain_neo4j import Neo4jGraph
from langchain_community.graphs import Neo4jGraph

load_dotenv()

True

In [61]:
url = os.getenv("NEO4J_URI")
username= os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")



graph = Neo4jGraph(
    url = url,
    username=username,
    password=password
)
# # Convert to GraphStore
# graph_store = GraphStore(
#     url=graph.url,
#     username=graph.username,
#     password=graph.password,
#     database=graph.database,  # if specified
# )


# graph_store

In [70]:
from langchain.chat_models import init_chat_model
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm = init_chat_model("gpt-4o")

# Initialize the graph transformer
# This utility will convert natural language text into a structured knowledge graph
transformer = LLMGraphTransformer(llm=llm, node_properties=["gender"]  # Extract these properties
)

In [71]:
text = """
Harry Potter and the Philosopher's Stone begins with young Harry Potter living with his awful aunt and uncle, the Dursleys, in their home at 4 Privet Drive. Harry's parents, James and Lily Potter, were killed by the dark wizard Lord Voldemort when Harry was just a baby. Mysteriously, when Voldemort tried to kill Harry with the Killing Curse, it rebounded and destroyed Voldemort's physical form instead, leaving Harry with only a lightning-bolt shaped scar on his forehead.

On his eleventh birthday, Harry discovers he is a wizard when Rubeus Hagrid, the Keeper of Keys and Grounds at Hogwarts School of Witchcraft and Wizardry, arrives to take him away from the Dursleys. Hagrid explains that Harry is famous in the wizarding world as "The Boy Who Lived" - the only person known to have survived the Killing Curse.

Harry travels to Diagon Alley, a hidden magical shopping district in London, where he purchases his school supplies including his wand - an eleven-inch holly wand with a phoenix feather core. Interestingly, the phoenix that provided the feather for Harry's wand also gave one other feather, which was used in Voldemort's wand, creating a mysterious connection between the two wizards.

At Hogwarts, Harry is sorted into Gryffindor House by the Sorting Hat, along with his new best friends Ron Weasley and Hermione Granger. Ron comes from a large wizarding family and has five older brothers - Bill, Charlie, Percy, Fred, and George - and a younger sister Ginny. Hermione is a Muggle-born witch, meaning both her parents are non-magical people, but she is exceptionally talented and becomes the brightest student in their year.

The trio soon discovers that something valuable is being hidden at Hogwarts - the Philosopher's Stone, created by Nicolas Flamel. This legendary stone can produce the Elixir of Life and transform any metal into pure gold. It's being protected by a series of magical challenges created by the Hogwarts professors, including a giant three-headed dog named Fluffy that belongs to Hagrid.

During their first year, the students attend various classes taught by memorable professors. Professor Minerva McGonagall teaches Transfiguration, Professor Severus Snape teaches Potions in the dungeons, and Professor Quirrell teaches Defense Against the Dark Arts. Snape appears to hate Harry from the first moment they meet, creating tension throughout the year.

Harry also discovers he has a natural talent for Quidditch, the wizarding sport played on broomsticks. He becomes the youngest Seeker in a century for the Gryffindor team, flying on a Nimbus 2000 broomstick given to him by Professor McGonagall. In Quidditch, the Seeker's job is to catch the Golden Snitch, a tiny winged ball that ends the game and earns 150 points.

The climax occurs when Harry, Ron, and Hermione realize that someone is trying to steal the Philosopher's Stone. They venture past Fluffy, through various magical obstacles including Devil's Snare (a plant defeated by Hermione's knowledge), a chamber of flying keys, and a life-sized wizard's chess game where Ron sacrifices himself so Harry can continue.

In the final chamber, Harry discovers that Professor Quirrell, not Snape as he suspected, has been working to steal the stone for Voldemort. Voldemort has been living on the back of Quirrell's head, hidden under his turban. When Quirrell tries to kill Harry, he discovers that he cannot touch Harry's skin - the protective magic from Lily Potter's sacrifice burns Quirrell when he makes contact.

Harry defeats Quirrell and prevents Voldemort from obtaining the stone, though Voldemort's spirit escapes. Albus Dumbledore, the wise Headmaster of Hogwarts, later explains to Harry that his mother's love created a powerful protection that saved him both as a baby and again in the chamber. Dumbledore also reveals that he and Nicolas Flamel have decided to destroy the Philosopher's Stone to prevent it from falling into the wrong hands.

The school year ends with the House Cup ceremony. Although Slytherin appears to have won, Dumbledore awards last-minute points to Harry, Ron, Hermione, and Neville Longbottom for their bravery, giving Gryffindor the victory. Harry returns to the Dursleys for the summer, but now he knows he has a place where he truly belongs - the magical world of Hogwarts.

Throughout the story, several other important characters play key roles. Draco Malfoy, a student from Slytherin House, becomes Harry's rival. Neville Longbottom, a forgetful but brave Gryffindor student, stands up to his friends at a crucial moment. Hagrid, despite being expelled from Hogwarts years ago, serves as a loyal friend and guide to Harry throughout his journey into the wizarding world.

The magical world contains many wondrous elements. Owls deliver mail, moving staircases connect the castle's many floors, portraits can talk and move between frames, and the Great Hall's ceiling is bewitched to look like the sky outside. Students learn to cast spells like Wingardium Leviosa for levitation and Alohomora to unlock doors. They study magical creatures, brew potions with ingredients like unicorn hair and dragon scales, and learn the history of magic from the ghost Professor Binns.

Platform Nine and Three-Quarters at King's Cross Station serves as the gateway to Hogwarts, accessed by walking through the barrier between platforms nine and ten. The Hogwarts Express, a scarlet steam engine, carries students from London to Hogsmeade Station, where they then travel to the castle by boat across the Black Lake or by horseless carriages.

The house system divides students into four houses: Gryffindor values bravery and courage, Hufflepuff values hard work and loyalty, Ravenclaw values intelligence and wit, and Slytherin values ambition and cunning. Each house has its own common room, dormitories, and ghost. Gryffindor's ghost is Nearly Headless Nick, while Slytherin's is the Bloody Baron.

This first adventure establishes the foundation for Harry's seven years at Hogwarts, introducing the magical world's rules, the ongoing threat of Voldemort, and the importance of friendship, courage, and love in overcoming dark forces. The themes of choice, identity, and the power of love continue throughout Harry's entire journey in the wizarding world.
"""

In [ ]:
# Wrap the input text in a LangChain Document object
# LangChain uses Document objects to represent text along with optional metadata
documents = [Document(page_content=text)]


# Convert the Document(s) into a structured knowledge graph
# graph_documents will contain nodes and relationships extracted from the text
graph_documents = transformer.convert_to_graph_documents(documents)
graph_documents

In [ ]:
graph.add_graph_documents(graph_documents)

In [ ]:
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from langchain_core.prompts import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """
Generate Cypher query to answer the question.

Rules:
- Use CONTAINS for name matching (handles partial names and typos)
- Use toLower() to ignore case
- Example: WHERE toLower(p.id) CONTAINS toLower("harry")

Schema:
{schema}

Question: {question}

Cypher Query:"""

cypher_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"]
)

# Create chain with schema awareness
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm, 
    graph=graph,
    verbose=True, 
     cypher_prompt=cypher_prompt,
    allow_dangerous_requests=True,
    validate_cypher=True  # This helps validate generated queries
)

In [ ]:
cypher_chain.invoke({
    "query": "who is mother of harry?"
})



> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


{'query': 'who is mother of harry?', 'result': "I don't know the answer."}

In [ ]:
# graph.query("MATCH (n) DETACH DELETE n")